##Installing Libraries

In [ ]:
!pip install streamlit yfinance plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
# pip install streamlit yfinance plotly tensorflow
import streamlit as st
from datetime import date, timedelta

import yfinance as yf
from plotly import graph_objs as go
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, LeakyReLU, Reshape, LSTM, Input
from tensorflow.keras.optimizers import Adam

START = "2015-01-01"
TODAY = date.today().strftime("%Y-%m-%d")

st.title('Stock Forecast App using GAN')

stocks = ('GOOG', 'AAPL', 'MSFT', 'GME')
selected_stock = st.selectbox('Select dataset for prediction', stocks)

n_years = st.slider('Years of prediction:', 1, 4)
period = n_years * 365

@st.cache
def load_data(ticker):
    data = yf.download(ticker, START, TODAY)
    data.reset_index(inplace=True)
    return data

data_load_state = st.text('Loading data...')
data = load_data(selected_stock)
data_load_state.text('Loading data... done!')

st.subheader('Raw data')
st.write(data.tail())

def plot_raw_data():
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data['Date'], y=data['Open'], name="stock_open"))
    fig.add_trace(go.Scatter(x=data['Date'], y=data['Close'], name="stock_close"))
    fig.layout.update(title_text='Time Series data with Rangeslider', xaxis_rangeslider_visible=True)
    st.plotly_chart(fig)

plot_raw_data()

# Prepare data for GAN
def create_dataset(dataset, time_step=1):
    X, Y = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        X.append(a)
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)

time_series_data = data[['Date', 'Close']]
time_series_data.set_index('Date', inplace=True)
time_series_data = time_series_data.rename(columns={"Close": "Value"})

time_step = 10
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(time_series_data['Value'].values.reshape(-1, 1))

X_train, y_train = create_dataset(scaled_data, time_step)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

# Build the generator
def build_generator():
    model = Sequential()
    model.add(Dense(100, input_dim=time_step))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(time_step, activation='tanh'))
    model.add(Reshape((time_step, 1)))
    return model

# Build the discriminator
def build_discriminator():
    model = Sequential()
    model.add(LSTM(50, input_shape=(time_step, 1)))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Build the generator
generator = build_generator()

# The generator takes noise as input and generates data
z = Input(shape=(time_step,))
generated_data = generator(z)

# For the combined model, we will only train the generator
discriminator.trainable = False

# The discriminator takes generated data as input and determines validity
validity = discriminator(generated_data)

# The combined model (stacked generator and discriminator)
combined = Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# Training the GAN
epochs = 1000
batch_size = 32
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

for epoch in range(epochs):
    # Train Discriminator
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_data = X_train[idx]
    noise = np.random.normal(0, 1, (batch_size, time_step))
    gen_data = generator.predict(noise)
    d_loss_real = discriminator.train_on_batch(real_data, valid)
    d_loss_fake = discriminator.train_on_batch(gen_data, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train Generator
    noise = np.random.normal(0, 1, (batch_size, time_step))
    g_loss = combined.train_on_batch(noise, valid)

    # Print progress
    if epoch % 1000 == 0:
        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100*d_loss[1]}] [G loss: {g_loss}]")

# Save models
generator.save('gan_generator.h5')
discriminator.save('gan_discriminator.h5')

2024-08-07 15:19:51.474 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new caching commands, `st.cache_data` or `st.cache_resource`.
More information [in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching logic used by `st.cache_data` and `st.cache_resource`.
This might lead to some problems or unexpected behavior in certain edge cases.

2024-08-07 15:19:51.475 No runtime found, using MemoryCacheStorageManager
[*********************100%%**********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/usr/local/lib/python3.10/dist-packages/keras/src/

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning:

The model does not have any trainable weights.



0 [D loss: 0.6707242727279663 | D accuracy: 95.3125] [G loss: [array(0.67525417, dtype=float32), array(0.67525417, dtype=float32), array(0.90625, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


ValueError: All arrays must be of the same length

In [ ]:
generator.save('gan_generator.h5')
discriminator.save('gan_discriminator.h5')

In [ ]:
import io, os, sys, setuptools, tokenize

In [ ]:
!pip install pystan==2.19.1.1
!pip install convertdate
!pip install lunarcalendar
!pip install holidays==0.10.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 42.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
^C
  Using cached convertdate-2.4.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached PyMeeus-0.5.12-py3-none-any.whl
Using cached convertdate-2.4.0-py3-none-any.whl (47 kB)
  Using cached LunarCalendar-0.0.9-py2.py3-none-any.whl.metadata (6.3 kB)
  Using cached ephem-4.1.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.0 kB)
Using cached LunarCalendar-0.0.9-py2.py3-none-any.whl (18 kB)
Using cached ephem-4.1.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.8 MB)
ERROR: Could not find a version that satisfies the requirement holidays==0.10.5 (from versions: 0.1, 0.2, 0.3, 0.3.1, 0.4, 0.4.1, 0.5, 0.6, 0.7, 0.8, 0.8.1, 0.9.3, 0.9.4, 0.9.5, 0.9.6, 0.9.7, 0.9.8, 0.9.9, 0.9.10, 0.9.11, 0.9.12, 0.10.1, 0.10.2, 0.10.3, 0.10.4, 0.10.5.2, 0.11.1, 0.11.2, 0.11.3.1, 0.12, 0.13, 0.14.2, 0.15, 0.16, 0.17, 0.17.2, 0.18, 0.19, 0.20, 0.21, 0.2

In [ ]:
!python -m pip install prophet

In [ ]:
import streamlit as st
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import date
from plotly import graph_objs as go
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
from prophet import Prophet
from prophet.plot import plot_plotly

START = "2015-01-01"
TODAY = date.today().strftime("%Y-%m-%d")

st.title('Stock Forecast App using Generative Adversarial Networks (GANs)')

stocks = ('GOOG', 'AAPL', 'MSFT', 'GME', 'TSLA')
selected_stock = st.selectbox('Select dataset for prediction', stocks)

n_years = st.slider('Years of prediction:', 1, 4)
period = n_years * 365

@st.cache_data
def load_data(ticker):
    data = yf.download(ticker, START, TODAY)
    data.reset_index(inplace=True)
    return data

data_load_state = st.text('Loading data...')
data = load_data(selected_stock)
data_load_state.text('Loading data... done!')

st.subheader('Raw data')
st.write(data.tail())

def plot_raw_data():
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data['Date'], y=data['Open'], name="stock_open"))
    fig.add_trace(go.Scatter(x=data['Date'], y=data['Close'], name="stock_close"))
    fig.layout.update(title_text='Time Series data with Rangeslider', xaxis_rangeslider_visible=True)
    st.plotly_chart(fig)

plot_raw_data()

# Prepare data for GAN
def create_dataset(dataset, time_step=1):
    X, Y = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        X.append(a)
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)

time_series_data = data[['Date', 'Close']]
time_series_data.set_index('Date', inplace=True)
time_series_data = time_series_data.rename(columns={"Close": "Value"})

time_step = 10
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(time_series_data['Value'].values.reshape(-1, 1))

X_train, y_train = create_dataset(scaled_data, time_step)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

# Load the models
generator = load_model('gan_generator.h5')
discriminator = load_model('gan_discriminator.h5')

# Make predictions
noise = np.random.normal(0, 1, (1, time_step))
generated_prediction = generator.predict(noise)
generated_prediction = generated_prediction.reshape(-1, 1)
generated_prediction = scaler.inverse_transform(generated_prediction)

st.subheader('Forecast data')
st.write(generated_prediction)

df_train = data[['Date', 'Close']]
df_train = df_train.rename(columns={"Date": "ds", "Close": "y"})

m = Prophet()
m.fit(df_train)
future = m.make_future_dataframe(periods=period)
forecast = m.predict(future)

# Show and plot forecast
st.subheader('Forecast data')
st.write(forecast.tail())

st.write(f'Forecast plot for {n_years} years')
fig1 = plot_plotly(m, forecast)
st.plotly_chart(fig1)

st.write("Forecast components")
fig2 = m.plot_components(forecast)
st.write(fig2)


In [ ]:
import streamlit as st
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import date
from plotly import graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
from prophet import Prophet
from prophet.plot import plot_plotly

START = "2015-01-01"
TODAY = date.today().strftime("%Y-%m-%d")

st.title('Stock Forecast App using Generative Adversarial Networks (GANs)')

# Data Collection
stocks = ('GOOG', 'AAPL', 'MSFT', 'GME', 'TSLA')
selected_stock = st.selectbox('Select dataset for prediction', stocks)

n_years = st.slider('Years of prediction:', 1, 4)
period = n_years * 365

@st.cache_data
def load_data(ticker):
    data = yf.download(ticker, START, TODAY)
    data.reset_index(inplace=True)
    return data

data_load_state = st.text('Loading data...')
data = load_data(selected_stock)
data_load_state.text('Loading data... done!')

# Exploratory Data Analysis (EDA)
st.subheader('Raw data')
st.write(data.tail())

# Line Graph
st.subheader('Line Graph')
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['Date'], y=data['Close'], name="Stock Close Price"))
fig.layout.update(title_text='Stock Close Price Over Time')
st.plotly_chart(fig)

# Scatter Plot
st.subheader('Scatter Plot')
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['Open'], y=data['Close'], mode='markers', name='Open vs Close'))
fig.layout.update(title_text='Open vs Close Price')
st.plotly_chart(fig)

# Histogram
st.subheader('Histogram')
fig = go.Figure()
fig.add_trace(go.Histogram(x=data['Close'], nbinsx=50, name='Close Price Distribution'))
fig.layout.update(title_text='Histogram of Close Prices')
st.plotly_chart(fig)

# QQ Plot
st.subheader('QQ Plot')
fig, ax = plt.subplots()
import scipy.stats as stats
stats.probplot(data['Close'].dropna(), dist="norm", plot=ax)
st.pyplot(fig)

# Descriptive Statistics
st.subheader('Descriptive Statistics')
st.write(data.describe())

# Dealing with Missing Values
if data.isnull().sum().sum() > 0:
    st.subheader('Dealing with Missing Values')
    st.write(data.isnull().sum())

    # Fill missing values
    data.fillna(method='ffill', inplace=True)
    st.write('Missing values handled.')

# Check Stationarity with ADF Test
st.subheader('Stationarity Check (ADF Test)')
def test_stationarity(series):
    result = adfuller(series)
    st.write(f'ADF Statistic: {result[0]}')
    st.write(f'p-value: {result[1]}')
    st.write(f'Critical Values: {result[4]}')

test_stationarity(data['Close'])

# If not stationary, perform transformations
if adfuller(data['Close'])[1] > 0.05:
    st.subheader('Transformations for Stationarity')
    # Example transformation: Log transformation
    data['Log_Close'] = np.log(data['Close'])
    test_stationarity(data['Log_Close'])

# Check Seasonality
st.subheader('Seasonality Decomposition')
result = seasonal_decompose(data['Close'], model='additive', period=365)
st.write('Seasonal Decomposition')
fig = result.plot()
st.pyplot(fig)

# ACF and PACF Plots
st.subheader('ACF and PACF Plots')
fig, ax = plt.subplots(2, 1, figsize=(10, 8))
plot_acf(data['Close'].dropna(), ax=ax[0])
plot_pacf(data['Close'].dropna(), ax=ax[1])
st.pyplot(fig)

# Identify Model Type
st.subheader('Model Identification')

# Load GAN Models
generator = load_model('gan_generator.h5')
discriminator = load_model('gan_discriminator.h5')

# Prepare data for GAN
def create_dataset(dataset, time_step=1):
    X, Y = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        X.append(a)
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)

time_series_data = data[['Date', 'Close']]
time_series_data.set_index('Date', inplace=True)
time_series_data = time_series_data.rename(columns={"Close": "Value"})

time_step = 10
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(time_series_data['Value'].values.reshape(-1, 1))

X_train, y_train = create_dataset(scaled_data, time_step)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

# Make predictions with GAN
noise = np.random.normal(0, 1, (1, time_step))
generated_prediction = generator.predict(noise)
generated_prediction = generated_prediction.reshape(-1, 1)
generated_prediction = scaler.inverse_transform(generated_prediction)

st.subheader('Forecast data using GAN')
st.write(generated_prediction)

# Forecasting with Prophet
df_train = data[['Date', 'Close']]
df_train = df_train.rename(columns={"Date": "ds", "Close": "y"})

m = Prophet()
m.fit(df_train)
future = m.make_future_dataframe(periods=period)
forecast = m.predict(future)

st.subheader('Forecast data')
st.write(forecast.tail())

st.write(f'Forecast plot for {n_years} years')
fig1 = plot_plotly(m, forecast)
st.plotly_chart(fig1)

st.write("Forecast components")
fig2 = m.plot_components(forecast)
st.write(fig2)

# Identify Lags and AIC/BIC Criteria
st.subheader('Identify Lags and Model Criteria')

# Identifying Lags using PACF
st.write('Lags identified using PACF plot.')

# AIC and BIC Criteria
# The following example is conceptual; for actual model fitting, you would calculate these for different models.
st.write('AIC and BIC criteria would be used to compare models. Lower values indicate a better fit.')


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.83.235.218


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.235.218:8501

your url is: https://soft-moose-listen.loca.lt
2024-08-07 16:30:07.119960: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-07 16:30:07.148714: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-07 16:30:07.157110: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-07 16:30:09.193340: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[*****************